In [ ]:
import pandas as pd
import numpy as np

from transformers import Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset, load_dataset, Audio

import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
DATASET = {
    "path": "PolyAI/minds14",
    "name": "cs-CZ",
    "split": None
}

SAMPLING_RATE = 16_000

WAV2VEC_MODEL_NAME = "/home/sulcm/.cache/models/wav2vec2/wav2vec2-cs-baseline/checkpoint-10000"

T5_MODEL_NAME = "t5-base"

In [ ]:
dataset = load_dataset(DATASET['path'], DATASET['name'], split=DATASET['split'])
dataset = dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))

In [ ]:
wav2vec_tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(WAV2VEC_MODEL_NAME)
wav2vec2_feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC_MODEL_NAME)
wav2vec_processor = Wav2Vec2Processor.from_pretrained(WAV2VEC_MODEL_NAME)
wav2vec_model = Wav2Vec2ForCTC.from_pretrained(WAV2VEC_MODEL_NAME)

In [ ]:
idx = 0
input_audio = dataset['train'][idx]['audio']
sentence = dataset['train'][idx]['transcription'].lower()

In [ ]:
inputs = wav2vec_processor(input_audio['array'], sampling_rate=SAMPLING_RATE, return_tensors='pt')

with torch.no_grad():
    logits = wav2vec_model(**inputs).logits

pred_ids = torch.argmax(logits, dim=-1)
transcription = wav2vec_tokenizer.batch_decode(pred_ids)

In [ ]:
transcription